<a href="https://colab.research.google.com/github/CGreenidge/GolfAIDataCleaning/blob/main/LifeBear_Data_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<ipython-input-16-f234e111f19a>:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)



Processing /content/lifebear_dataset_chunk_1.csv...
An error occurred while changing the date format in date_of_birth: time data "1984-11-09" doesn't match format "%d/%m/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

DataFrame after changing the date format:
   id    login_id                     email                          password  \
0   1    sugimoto   sugimoto@lifebear.co.jp  f0bac04aa1b45cf443d722d6f71c0250   
1   2         kou  nakanishi@lifebear.co.jp  48207c322ee5bb156ffec9f08c960aaa   
2   3      yusuke     yuozawa1208@gmail.com  048261a8024ce51d379eb53cc51aaf33   
3   4  entyan1106        endo1106@gmail.com  cd77a9dac26260a104facda5665eb3ab   
4   5      kurik

In [ ]:
import pandas as pd
import os
import re

def load_csv_to_dataframe(file_path):
    """Load a CSV file into a DataFrame."""
    try:
        if not os.path.isfile(file_path):
            print(f"File {file_path} does not exist. Please check the file path.")
            return None
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"An error occurred while loading {file_path}: {e}")
        return None

def drop_duplicates_by_email(df, keep='first'):
    """Drop duplicate rows based on the 'email' column."""
    if df is not None and 'email' in df.columns:
        return df.drop_duplicates(subset='email', keep=keep)
    return df

def change_date_format(df, column_name):
    """Change the date format in the specified column from 'day/month/year' to 'month/day/year'."""
    if df is not None and column_name in df.columns:
        try:
            df[column_name] = pd.to_datetime(df[column_name], format='%d/%m/%Y').dt.strftime('%m/%d/%Y')
        except Exception as e:
            print(f"An error occurred while changing the date format in {column_name}: {e}")
    return df

def fill_nan_values(df, fill_value=None):
    """Fill NaN values in the DataFrame with a specified value or method."""
    if df is not None:
        return df.fillna(fill_value)
    return df

def fill_missing_date_of_birth_and_gender(df, date_of_birth_fill='01/01/1900', gender_fill='Not Specified'):
    """Fill missing values in the 'date_of_birth' and 'gender' columns."""
    if df is not None:
        df['date_of_birth'].fillna(date_of_birth_fill, inplace=True)
        df['gender'].fillna(gender_fill, inplace=True)
    return df

def rename_columns(df, column_mapping):
    """Rename specified columns in the DataFrame."""
    return df.rename(columns=column_mapping) if df is not None else df

def is_valid_email(email):
    """Check if the provided email is valid."""
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(email_regex, email) is not None

def append_to_garbage_file(invalid_emails_df, garbage_file='garbage_file.csv'):
    """Append invalid email entries to the garbage file."""
    if not invalid_emails_df.empty:
        invalid_emails_df.to_csv(garbage_file, index=False, mode='a', header=not os.path.exists(garbage_file))
        print(f"Invalid emails saved to {garbage_file}.")

def filter_valid_emails(df, email_column, garbage_file='garbage_file.csv'):
    """Filter valid emails and save invalid entries to a garbage file."""
    if df is not None and email_column in df.columns:
        valid_emails_df = df[df[email_column].apply(is_valid_email)]
        invalid_emails_df = df[~df[email_column].apply(is_valid_email)]

        # Append invalid emails to garbage file
        append_to_garbage_file(invalid_emails_df, garbage_file)

        return valid_emails_df
    return df

def drop_columns(df, columns_to_drop):
    """Drop specified columns from the DataFrame."""
    return df.drop(columns=columns_to_drop, errors='ignore') if df is not None else df

def clean_file(file_path, output_file='cleaned_life_bear_data.csv'):
    """Clean a single file and export the cleaned data."""
    dataframe = load_csv_to_dataframe(file_path)

    if dataframe is not None:
        print(f"\nProcessing {file_path}...")

        # Rename columns
        renamed_data = rename_columns(dataframe, {
            'birthday_on': 'date_of_birth',
            'mail_address': 'email'
        })

        # Change date format
        updated_data = change_date_format(renamed_data, 'date_of_birth')
        print("\nDataFrame after changing the date format:")
        print(updated_data.head())

        # Fill missing values in date_of_birth and gender
        filled_data = fill_missing_date_of_birth_and_gender(updated_data)
        print("\nDataFrame after filling missing values in date_of_birth and gender:")
        print(filled_data.head())

        # Filter valid emails
        valid_emails_data = filter_valid_emails(filled_data, 'email')
        print("\nDataFrame after filtering valid emails:")
        print(valid_emails_data.head())

        # Drop unnecessary columns
        final_data = drop_columns(valid_emails_data, ['id', 'created_on'])
        print("\nFinal DataFrame after dropping unnecessary columns:")
        print(final_data.head())

        # Drop duplicates based on email
        cleaned_data_by_email = drop_duplicates_by_email(final_data)
        print("\nDataFrame after dropping duplicates based on email:")
        print(cleaned_data_by_email.head())

        # Export cleaned data to a CSV file
        cleaned_data_by_email.to_csv(output_file, index=False)
        print(f"Cleaned data exported to {output_file}.")

        return cleaned_data_by_email  # Return cleaned data for merging

def merge_cleaned_data(cleaned_files):
    """Merge all cleaned DataFrames into one and return it."""
    merged_df = pd.concat(cleaned_files, ignore_index=True)
    return merged_df

def main(file_paths):
    """Main workflow function to clean multiple files and merge cleaned data."""
    cleaned_data_frames = []

    for i, file_path in enumerate(file_paths):
        output_file = f'cleaned_life_bear_data_{i+1}.csv'  # Unique output file for each input
        cleaned_data = clean_file(file_path, output_file)
        if cleaned_data is not None:
            cleaned_data_frames.append(cleaned_data)  # Collect cleaned data for merging

    # Merge all cleaned data
    if cleaned_data_frames:
        merged_data = merge_cleaned_data(cleaned_data_frames)
        merged_output_file = 'merged_cleaned_life_bear_data.csv'
        merged_data.to_csv(merged_output_file, index=False)
        print(f"\nAll cleaned data merged and exported to {merged_output_file}.")

# Example usage
file_paths = [
    r"/content/lifebear_dataset_chunk_1.csv",
    r"/content/lifebear_dataset_chunk_2.csv",
    r"/content/lifebear_dataset_chunk_3.csv",
    r"/content/lifebear_dataset_chunk_4.csv"
]
main(file_paths)


<ipython-input-17-d98e1b588a15>:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
<ipython-input-17-d98e1b588a15>:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date_of_birth'].fillna(date_of_birth_fill, inplace=True)



Processing /content/lifebear_dataset_chunk_1.csv...
An error occurred while changing the date format in date_of_birth: time data "1984-11-09" doesn't match format "%d/%m/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

DataFrame after changing the date format:
   id    login_id                     email                          password  \
0   1    sugimoto   sugimoto@lifebear.co.jp  f0bac04aa1b45cf443d722d6f71c0250   
1   2         kou  nakanishi@lifebear.co.jp  48207c322ee5bb156ffec9f08c960aaa   
2   3      yusuke     yuozawa1208@gmail.com  048261a8024ce51d379eb53cc51aaf33   
3   4  entyan1106        endo1106@gmail.com  cd77a9dac26260a104facda5665eb3ab   
4   5      kurik

<ipython-input-17-d98e1b588a15>:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['gender'].fillna(gender_fill, inplace=True)
<ipython-input-17-d98e1b588a15>:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Specified' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['gender'].fillna(gender_fill, inplace=True)


Invalid emails saved to garbage_file.csv.

DataFrame after filtering valid emails:
   id    login_id                     email                          password  \
0   1    sugimoto   sugimoto@lifebear.co.jp  f0bac04aa1b45cf443d722d6f71c0250   
1   2         kou  nakanishi@lifebear.co.jp  48207c322ee5bb156ffec9f08c960aaa   
2   3      yusuke     yuozawa1208@gmail.com  048261a8024ce51d379eb53cc51aaf33   
3   4  entyan1106        endo1106@gmail.com  cd77a9dac26260a104facda5665eb3ab   
4   5      kuriki          kuriki@wavy4.com  a026597c294cc48cd20ae361f10cbab1   

            created_at          salt date_of_birth gender  
0  2012-01-13 22:54:05  yGwBKynnsctI    1984-11-09    0.0  
1  2012-01-14 12:48:31  aha6EuRYCDvU    1986-11-13    0.0  
2  2012-01-17 15:33:22  PVS59dPWk9BH    1984-12-08    0.0  
3  2012-01-17 15:37:02  vLZI6TVCJowN    1987-11-06    0.0  
4  2012-01-17 18:52:32  swFznWWk79fg    1986-10-21    0.0  

Final DataFrame after dropping unnecessary columns:
     login_id    

<ipython-input-17-d98e1b588a15>:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date_of_birth'].fillna(date_of_birth_fill, inplace=True)
<ipython-input-17-d98e1b588a15>:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

Invalid emails saved to garbage_file.csv.

DataFrame after filtering valid emails:
        id      login_id                         email  \
0  1065670  classical236      o9o27566991@docomo.ne.jp   
1  1065674      u2makoto            U2makoto@gmail.com   
2  1065676    horiuchi37          horiuchi37@gmail.com   
3  1065677       masatoi               imasato@mac.com   
4  1065678   nakkamu0127  bb.01-27.as-an@i.softbank.jp   

                           password           created_at          salt  \
0  2dfc3c797e50090f5088dbfec5175013  2013-10-12 10:43:34  yuE86D7DSdqc   
1  409f7434dbeba141eaacad0b22534ead  2013-10-12 10:43:50  PMUfVzsWmPHp   
2  a531937e15556e3c69e3fec4785620b7  2013-10-12 10:44:32  zAjryhQWmG4b   
3  bc4bd44d7f3b1dfeebbaac779a075a73  2013-10-12 10:44:34  VVt1Zo0dCj2r   
4  91212a117ebeda44d5d94816c751d3a7  2013-10-12 10:44:38  7PxXj4JX4HM4   

  date_of_birth         gender  
0    01/01/1900  Not Specified  
1    01/01/1900            0.0  
2    01/01/1900  Not Spe

<ipython-input-17-d98e1b588a15>:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date_of_birth'].fillna(date_of_birth_fill, inplace=True)
<ipython-input-17-d98e1b588a15>:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

Invalid emails saved to garbage_file.csv.

DataFrame after filtering valid emails:
        id       login_id                        email  \
0  3613754   yuuryoumama9       yuuryoumama9@gmail.com   
1  3613759  miyukkuma1010      miyukkuma1010@gmail.com   
2  3613760        yaataka   H.t.1223rock@i.softbank.jp   
3  3613762      funashyyy  tvxq0409mako@softbank.ne.jp   
4  3613763       masayasi       misobe2558@ezweb.ne.jp   

                           password           created_at          salt  \
0  32bc513b13f95524e94c2d0b8b94406d  2014-12-28 19:29:33  a2ID4CUKBzJk   
1  574788a53114a83a0137960df6655173  2014-12-28 19:30:06  NloBaM4aQSge   
2  6da4856a41f6c0a4c532e402c9e3e6a6  2014-12-28 19:30:10  6FWAFXSW5LWB   
3  8f9480bc3bcf22f34a4b3df6545076ee  2014-12-28 19:30:38  9tW3EdXI2smp   
4  e9b153f9c3860cdefeb718d9bfbd35e0  2014-12-28 19:30:43  A8DNljm2mlgs   

  date_of_birth         gender  
0    01/01/1900            1.0  
1    01/01/1900            1.0  
2    01/01/1900  Not Spe

<ipython-input-17-d98e1b588a15>:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date_of_birth'].fillna(date_of_birth_fill, inplace=True)
<ipython-input-17-d98e1b588a15>:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

Invalid emails saved to garbage_file.csv.

DataFrame after filtering valid emails:
        id        login_id                             email  \
0  6923270        yuxibear              neige.q126@gmail.com   
1  6923276     smile12nino             kie.nino5@ezweb.ne.jp   
2  6923280        yuriimai          uxipi-0jat@i.softbank.jp   
3  6923283  toyonagahiromi  toyoshin-mamamail@softbank.ne.jp   
4  6923286        mmkk1121         mm-melody-kk@docomo.ne.jp   

                           password           created_at          salt  \
0  6294784224ea814d5a0e58bc61f603bc  2016-07-03 22:35:35  J0eYeAd8QdvI   
1  0a9446606dabbc02e979be3aa2099574  2016-07-03 22:36:19  uhMUbiNEt8tN   
2  42f8fa01b3d252610ed519527816cf2b  2016-07-03 22:36:48  m1Ph4vINFazr   
3  a7b07f56cf0152432efd8da90f42d14c  2016-07-03 22:37:04  8juVFoWdHpe9   
4  34aaf7de58f4804ba21a308ecc7205cc  2016-07-03 22:37:16  e28wCo0cmgug   

  date_of_birth         gender  
0    01/01/1900  Not Specified  
1    01/01/1900  Not 